## Real-Time Emotion Detection with Webcam

### 1.Import the Libraries

In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [4]:
# load the model
model = load_model('emotion_model.keras')

# Emotion labels (FER2013 dataset has 7 classes)
emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

# Load Face Detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")

# Capture Video
cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
        
        # Extract face region
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48,48))   # resize to model input
        roi_gray = roi_gray.astype("float") / 255.0
        roi_gray = np.expand_dims(roi_gray, axis=-1)  # add channel dimension
        roi_gray = np.expand_dims(roi_gray, axis=0)   # add batch dimension
        
        # Predict emotion
        preds = model.predict(roi_gray, verbose=0)
        emotion = emotion_labels[np.argmax(preds)]
        # Put text
        cv2.putText(frame, emotion, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
    # Show video
    cv2.imshow("Emotion Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()